In [12]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import re


In [29]:
#url de libros Uptc
url = 'https://editorial.uptc.edu.co/catalogo.html?product_list_limit=60'
bucle = 'https://editorial.uptc.edu.co/catalogo.html?p=2&product_list_limit=60'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html')


scrap = soup.find('div',class_="sa-product-container-v3 grid")
productos_url = scrap.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

url_libros = [url.find('a')["href"] for url in productos_url]
urls = pd.DataFrame(url_libros,columns=['url'])
for i in range(2,6):
    url = f'https://editorial.uptc.edu.co/catalogo.html?p={i}&product_list_limit=60'
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html')
    productos_url = soup.find('div',class_="sa-product-container-v3 grid").find_all('li',class_="sa-product-item-v3 sa-product_grid_size")
    url_libros = [url.find('a')["href"] for url in productos_url]
    urls = pd.concat([urls,pd.DataFrame(url_libros,columns=['url'])])



In [6]:
url_libros = pd.read_csv('urls.csv')

In [3]:
url_libros.shape

(298, 1)

In [8]:
datos = pd.read_json('datos.json')
datos.head()

,title,colaboradores,precio,palabras clave,doi,descripcion,metadata_ebook,metadata_imp
0,[Transporte de fluidos bombas centrífugas],"[[Otto Caro Niño, Orlando Díaz Parra]]","[[, COP $40.000, COP $30.000]]","[[Fluido, Sistema de bombeo, Bomba centrífuga,...",10.19053/uptc.9789586609265,[\nUn sistema compuesto por una conducción y s...,"{'Detalle de formato:': 'Digital: descarga', '...","{'DOI:': 'Tapa blanda o Bolsillo', 'Detalle de..."
1,[Introducción al cálculo de probabilidades par...,[[Alex L. Rojas]],[[]],"[[Espacio muestral, Probabilidad, Probabilidad...",Función cuantil,[\nEl propósito de este libro es servir de tex...,{'Detalle de formato:': 'Tapa blanda o Bolsill...,None
2,[Derechos colectivos en Colombia:],"[[Luis Bernardo Díaz Gamboa, Pedro Alfonso Sán...",[[]],"[[Derecho colectivo, Política pública, Estado ...",Planes de desarrollo,[\nLos derechos colectivos permiten una adecua...,{'Detalle de formato:': 'Tapa blanda o Bolsill...,None
3,[Resistencias y voces de las mujeres:],"[[Celina de Jesús Trimiño Velásquez, Claire Ta...","[[, COP $40.000]]","[[Conflicto armado colombiano, Luchas y voces ...",10.19053/uptc.9789586609388,[\nEl libro ofrece una visión integral de algu...,"{'DOI:': 'Tapa blanda o Bolsillo', 'Detalle de...",None
4,[Manual de prácticas pedagógicas para la forma...,[[Myriam Cecilia Leguizamón González]],[[]],"[[Práctica pedagógica, Maestro, Pedagogía, For...",Educación,[\nEste manual está diseñado para docentes de ...,{'Detalle de formato:': 'Tapa blanda o Bolsill...,None


In [53]:
#urls.to_json('urls.json',index=False)
urls.shape

(298, 1)

In [13]:

def extraer_palabras_clave(texto):
    # Patrón para encontrar palabras clave y sus valores
    patron = r'([^:]+):\s*([^\n]+)'
    
    # Encontrar todas las coincidencias
    coincidencias = re.findall(patron, texto)
    
    # Crear diccionario con las palabras clave
    palabras_clave = {}
    for clave, valor in coincidencias:
        # Limpiar espacios en blanco
        clave = clave.strip()
        valor = valor.strip()
        palabras_clave[clave] = valor
    
    return palabras_clave

In [16]:
temp_datos = []
for i in range(1,20):
    titulos = []
    autores = []
    palabras_clave = []
    precios = []
    descripcion = []


    res = requests.get(url_libros['url'][i])
    soup = BeautifulSoup(res.text,'html')
    time.sleep(5)
    scrap=soup.find('div',class_="product-info-main")
    titulos.append(scrap.find('div',class_="col-xs-12").find('h2').text)
    ## busquemos lo autores
    temp = [i for i in scrap.find_all('div',class_="col-xs-12")]
    temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_author' in a['href'] ]
    autores.append(temp1)
    # busca las palabras clave
    temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' in a['href'] ]
    palabras_clave.append(temp1)
    # DOi
    try:
        temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' not in a['href'] and  'products_author' not in a['href']]
        if temp1 != []:
            doi = temp1
        else :
            doi = None 
    except Exception as e:
        print(f'No se encontro doi: {e}')
        doi = None

    # precios
    try:
        temp = [a for a in scrap.find_all('div',class_='price-box price-final_price')]
        temp1 = [a.text.strip() for a in temp]
        precios.append(temp1)
    except Exception as e:
        print(f'No se encontro precio: {e}')
        precios.append('Gratuito')
    # descripcion
    descripcion.append(scrap.find('div',class_='description-wrapper').text)
    # eBook
    try:
        scrap = soup.find('div',class_="product-info-metadata")
        metadata_ebook = extraer_palabras_clave(scrap.text)
    except Exception as e:
        print(f'No se encontro metadata_ebook: {e}')
        metadata_ebook = None
    #impreso
    try:
        scrap = soup.find_all('div',class_="product-info-metadata")[1]
        metadata_imp = extraer_palabras_clave(scrap.text)
    except Exception as e:
        print(f'No se encontro metadata_imp: {e}')
        metadata_imp = None
    datos = {
            "title": titulos,
            "colaboradores":autores,
            "precio":precios,
            "palabras clave":palabras_clave,
            "doi":doi,
            "descripcion":descripcion,
            "metadata_ebook":metadata_ebook,
            "metadata_imp":metadata_imp
        }
    temp_datos.append(datos)
    

No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range
No se encontro metadata_imp: list index out of range


In [19]:
with open('datos.json', 'w', encoding='utf-8') as f:
    json.dump(temp_datos, f, indent=4, ensure_ascii=False)


In [ ]:
scrap = soup.find_all('div',class_="product-info-metadata")[1]
print(scrap.text)






DOI:
doi.org/10.19053/uptc.9789586609265 

Detalle de formato:
Tapa blanda o Bolsillo


Estado:
Activo


ISBN-13: 9789586609265


Tipo de contenido principal:
Texto (legible a simple vista)


Idioma del texto:
Español


Tamaño (cm):
14 x 21  x 0.6


Número de páginas del contenido principal:106 Páginas


Sello editorial: universidad Pedagógica y Tecnológica de Colombia
Tipo de edición: Nueva edición
Número de edición: 1

Ciudad de publicación:
Tunja


País de publicación:
Colombia


Fecha de publicación: 2025

Tipo de restricción de venta: Exclusivo para un punto o canal de venta

SIIEL SKU: 419307

Precio: COP $40.000






In [11]:
import re

def extraer_palabras_clave(texto):
    # Patrón para encontrar palabras clave y sus valores
    patron = r'([^:]+):\s*([^\n]+)'
    
    # Encontrar todas las coincidencias
    coincidencias = re.findall(patron, texto)
    
    # Crear diccionario con las palabras clave
    palabras_clave = {}
    for clave, valor in coincidencias:
        # Limpiar espacios en blanco
        clave = clave.strip()
        valor = valor.strip()
        palabras_clave[clave] = valor
    
    return palabras_clave

extraer_palabras_clave(scrap.text)

{'DOI': 'doi.org/10.19053/uptc.9789586609265',
 'Detalle de formato': 'Tapa blanda o Bolsillo',
 'Estado': 'Activo',
 'ISBN-13': '9789586609265',
 'Tipo de contenido principal': 'Texto (legible a simple vista)',
 'Idioma del texto': 'Español',
 'Tamaño (cm)': '14 x 21  x 0.6',
 'Número de páginas del contenido principal': '106 Páginas',
 'Sello editorial': 'universidad Pedagógica y Tecnológica de Colombia',
 'Tipo de edición': 'Nueva edición',
 'Número de edición': '1',
 'Ciudad de publicación': 'Tunja',
 'País de publicación': 'Colombia',
 'Fecha de publicación': '2025',
 'Tipo de restricción de venta': 'Exclusivo para un punto o canal de venta',
 'SIIEL SKU': '419307',
 'Precio': 'COP $40.000'}

In [ ]:
scrap = soup.find_all('div',class_="product-info-metadata")[1]
print(scrap.text)






DOI:
doi.org/10.19053/uptc.9789586609265 

Detalle de formato:
Tapa blanda o Bolsillo


Estado:
Activo


ISBN-13: 9789586609265


Tipo de contenido principal:
Texto (legible a simple vista)


Idioma del texto:
Español


Tamaño (cm):
14 x 21  x 0.6


Número de páginas del contenido principal:106 Páginas


Sello editorial: universidad Pedagógica y Tecnológica de Colombia
Tipo de edición: Nueva edición
Número de edición: 1

Ciudad de publicación:
Tunja


País de publicación:
Colombia


Fecha de publicación: 2025

Tipo de restricción de venta: Exclusivo para un punto o canal de venta

SIIEL SKU: 419307

Precio: COP $40.000






In [19]:
temp_datos[0]['metadata_imp']

{'DOI:': 'Tapa blanda o Bolsillo',
 'Detalle de formato:': 'Activo',
 'Estado:': '9789586609265',
 'ISBN-13:': 'Texto (legible a simple vista)',
 'Tipo de contenido principal:': 'Español',
 'Idioma del texto:': '14 x 21  x 0.6',
 'Tamaño (cm):': '106 Páginas',
 'Número de páginas del contenido principal:': 'universidad Pedagógica y Tecnológica de Colombia',
 'Sello editorial: ': 'Nueva edición',
 'Tipo de edición: ': '1',
 'Número de edición: ': 'Tunja',
 'Ciudad de publicación:': 'Colombia',
 'País de publicación:': '2025',
 'Fecha de publicación:': 'Exclusivo para un punto o canal de venta',
 'Tipo de restricción de venta:': '419307',
 'SIIEL SKU:': 'COP $40.000'}

In [53]:
scrap = soup.find('div',class_="product-info-metadata")
temp1 = [a.text for a in scrap.find_all('span')]


['Digital: descarga y online', 'PDF', 'Activo', '9789586609449', 'Texto (legible a simple vista)', 'DRM', 'Español', 'Universidad Pedagógica y Tecnológica de Colombia', 'Nueva edición', '1', 'Tunja', 'Colombia', '2025', 'Exclusivo para un punto o canal de venta']


In [47]:
[]==[]

True

In [52]:
print(soup.find('div',class_='product data items').find('div',{'class':'item content'}).text.replace('\n',''))

BISACPHI000000 FILOSOFÍA > GeneralTHEMAQDT Temas de la filosofíaDEWEY107 Filosofía y psicología > Generalidades > Educación, investigación en filosofía


In [58]:
temp_datos[-1]

{'title': ['Gestión Educativa, Perspectivas, Actores y Contextos'],
 'colaboradores': [['Liliana Inés Ávila Garzón',
   'Rafael Enrique Buitrago Bonilla',
   'María Rosa Estupiñán Aponte',
   'Adriana María Cárdenas Casas',
   'Alex Oswaldo Sánchez Huarcaya',
   'Alexandra Inés Puentes Suárez',
   'Angie Lorena Ariza Velasco',
   'Diana Margot Guerrero Galarza',
   'Esmeralda Díaz López',
   'Kassandra Verastegui Vives',
   'Nidia Marcela García Gómez',
   'Ruby Paola Villavicencio Gastelu',
   'Ruth Nayibe Cárdenas Soler',
   'Sandra Milena Pinzón Jaime']],
 'precio': [['', 'COP $70.000']],
 'palabras clave': [['Gestión Educativa',
   'Emociones',
   'Maestros',
   'Memorias de Gestores',
   'Cultura Escolar',
   'Contextos']],
 'doi': '10.19053/uptc.9789586608350',
 'descripcion': ['\nGestión Educativa. Perspectivas, Actores y Contextos es un libro imprescindible para aquellos interesados en ahondar en los procesos de gestión en el ámbito educativo. A través de los distintos capítulo

In [53]:
temp1 = [a.text for a in scrap.find_all('strong')]
temp2 = [a.text for a in scrap.find_all('span')]
metadata_imp ={c:v for c,v in zip(temp1,temp2)}

In [54]:
metadata_imp

{'DOI:': 'Tapa blanda o Bolsillo',
 'Detalle de formato:': 'Activo',
 'Estado:': '9789586609265',
 'ISBN-13:': 'Texto (legible a simple vista)',
 'Tipo de contenido principal:': 'Español',
 'Idioma del texto:': '14 x 21  x 0.6',
 'Tamaño (cm):': '106 Páginas',
 'Número de páginas del contenido principal:': 'universidad Pedagógica y Tecnológica de Colombia',
 'Sello editorial: ': 'Nueva edición',
 'Tipo de edición: ': '1',
 'Número de edición: ': 'Tunja',
 'Ciudad de publicación:': 'Colombia',
 'País de publicación:': '2025',
 'Fecha de publicación:': 'Exclusivo para un punto o canal de venta',
 'Tipo de restricción de venta:': '419307',
 'SIIEL SKU:': 'COP $40.000'}

In [44]:
metadata_ebook

{'Detalle de formato:': 'Digital (suministrado electrónicamente)',
 'Formato:': 'EPUB',
 'Estado:': 'Activo',
 'ISBN-13:': '9789586609272',
 'Tipo de contenido del producto:': 'Texto (legible a simple vista)',
 'Protección técnica de publicación digital:': 'DRM',
 'Tipo de restricción de uso en publicaciones digitales:': 'Share',
 'Permiso de uso en publicaciones digitales:': 'Permitido, con limitaciones',
 'Idioma del texto:': 'Español',
 'Tamaño del archivo:': '1,593 Kbytes',
 'Sello editorial: ': 'Universidad Pedagógica y Tecnológica de Colombia',
 'Tipo de edición: ': 'Nueva edición',
 'Número de edición: ': '1',
 'Ciudad de publicación:': 'Tunja',
 'País de publicación:': 'Colombia',
 'Fecha de publicación:': '2025',
 'Tipo de restricción de venta:': 'Exclusivo para un punto o canal de venta',
 'Precio: ': 'COP $30.000'}

In [41]:
len(temp1) ,len(temp2)

(18, 18)

In [39]:
for a in scrap.find_all('span'):
    print(a.text)

Digital (suministrado electrónicamente)
EPUB
Activo
9789586609272
Texto (legible a simple vista)
DRM
Share
Permitido, con limitaciones
Español
1,593 Kbytes
Universidad Pedagógica y Tecnológica de Colombia
Nueva edición
1
Tunja
Colombia
2025
Exclusivo para un punto o canal de venta
COP $30.000


In [38]:
temp

['Detalle de formato:',
 'Formato:',
 'Estado:',
 'ISBN-13:',
 'Tipo de contenido del producto:',
 'Protección técnica de publicación digital:',
 'Tipo de restricción de uso en publicaciones digitales:',
 'Permiso de uso en publicaciones digitales:',
 'Idioma del texto:',
 'Tamaño del archivo:',
 'Sello editorial: ',
 'Tipo de edición: ',
 'Número de edición: ',
 'Ciudad de publicación:',
 'País de publicación:',
 'Fecha de publicación:',
 'Tipo de restricción de venta:',
 'Precio: ']

In [115]:
temp1

['Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento']

In [73]:
temp1

['Otto Caro Niño',
 'Orlando Díaz Parra',
 'Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento',
 '10.19053/uptc.9789586609265']

In [9]:
res.status_code

200

 tenemos un estatus 200 lo cual es correcto la solicitud
 

In [4]:
soup = BeautifulSoup(res.text,'html')

In [19]:
soup.find('title').get_text()

'Catálogo - UPTC'

In [22]:
datos=soup.find('div',class_="sa-product-container-v3 grid")


In [26]:
productos = datos.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

In [ ]:
a = {
    titulo: 
    colaboradores:
    ano:
    precio:
}

In [48]:
title =[i.find('span').get_text(strip=True) for i in productos]

In [62]:
ano = [i.find('div',class_='product-list-edition').get_text(strip=True) for i in productos]

In [76]:
productos[].find('span',class_='minimal-price').get_text()

'\nImpreso\n\n\nCOP $40.000\n\n\n'

In [101]:
datos = []
for i in productos:
    a = {
        "title":i.find('span').get_text(strip=True),
        "colaboradores":list(i.find('div',class_='sa-product-author-v3').stripped_strings)[1],
        "precio": i.find('span',class_='minimal-price').get_text().replace("\n",""),
        "ano": i.find('div',class_='product-list-edition').get_text(strip=True)
    }
    datos.append(a)


In [102]:
datos

[{'title': 'Transporte de fluidos bombas centrífugas',
  'colaboradores': 'Otto Caro Niño                                                                                     y otros',
  'precio': 'eBookCOP $30.000',
  'ano': '2025'},
 {'title': 'Introducción al cálculo de probabilidades para estudiantes de ingeniería',
  'colaboradores': 'Alex L. Rojas',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Derechos colectivos en Colombia:',
  'colaboradores': 'Luis Bernardo Díaz Gamboa                                                                                     y otros',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Resistencias y voces de las mujeres:',
  'colaboradores': 'Celina de Jesús Trimiño Velásquez                                                                                     y otros',
  'precio': 'ImpresoCOP $40.000',
  'ano': '2025'},
 {'title': 'Manual de prácticas pedagógicas para la formación de maestros.',
  'colaboradores': 'Myriam Cecilia

## 1.2 Encontrando las url de cada producto.

+ esto nos permitira acceder a toda la informacion del producto 

In [ ]:
productos = soup.find('div',class_='sa-product-container-v3 grid').find_all('li')
urls=[url.find('a') ['href'] for url in productos]

In [18]:
len(productos)

68

In [31]:
productos[0].find('a')

<a href="https://editorial.uptc.edu.co/gpd-transporte-de-fluidos-bombas-centrifugas-9789586609265-67ebeccc7db73.html">
<img alt="Portada de la publicación Transporte de fluidos bombas centrífugas" src="https://simehbucket.s3.amazonaws.com/images/415ebe2115322046c17eea71821248a5-medium.jpg"/>
</a>